In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel
# import bitsandbytes as bnb
import os
from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
# from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm

In [ ]:
train_data = pd.read_csv('/content/train.csv')

In [ ]:
# Create empty lists to store your data
ids = []
categories = []
questions = []
answers = []

# Iterate through rows of the DataFrame
for index, row in train_data.iterrows():
    # Iterate through '질문' columns
    for q_col in ['질문_1', '질문_2']:
        # Iterate through '답변' columns
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # Combine question and answer with </s> token
            input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            # Tokenize the combined text
            input_ids = tokenizer.encode(input_text, return_tensors='pt')

            # Append id, category, question, and answer to the lists
            ids.append(row['id'])
            categories.append(row['category'])
            questions.append(row[q_col])
            answers.append(row[a_col])

# Create a new DataFrame with the combined data
new_df = pd.DataFrame({'id': ids, 'category': categories, 'Question': questions, 'Answer': answers})

In [ ]:
new_df.head(30)

,id,category,Question,Answer
0,TRAIN_000,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,TRAIN_000,건축구조,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,TRAIN_000,건축구조,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,TRAIN_000,건축구조,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,TRAIN_000,건축구조,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
5,TRAIN_000,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
6,TRAIN_000,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
7,TRAIN_000,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
8,TRAIN_000,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
9,TRAIN_000,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...


In [ ]:
del new_df['id']
del new_df['category']
new_df.head()

,Question,Answer
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...


In [ ]:
len(new_df)

6440

In [ ]:
df_df = pd.read_excel('/content/답변추가_통합_수정본.xlsx', index_col='NO')

In [ ]:
df_df.head(60)

,카테고리,질문1,질문2,답변1,답변2,답변3,답변4,답변5
NO,,,,,,,,
1,가구수정,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,벽지를 붙일 때 정확한 위치를 어떻게 찾나요?,"벽지를 정확한 위치에 붙이기 위해선 먼저 벽의 수평과 수직을 확인하고, 시작점을 ...",벽지를 붙일 때는 벽지의 패턴이나 디자인이 맞도록 조심스럽게 배치하는 것이 중요합니...,벽지를 부착할 때는 시작점을 정한 후에 벽의 수평과 수직을 확인하는 것이 중요합니다...,벽지를 부착할 때는 먼저 벽의 수평과 수직을 정확히 확인한 후에 시작점을 결정해야 ...,벽지를 붙이기 전에는 레이저 수평계나 수평 앱을 사용하여 벽의 수평과 수직을 정밀하...
2,가구수정,벽지 위치를 잘못 붙였을 때 어떻게 고치지?,벽지 위치를 잘못 붙였을 때 어떻게 수정하나요?,"벽지를 잘못 붙인 경우, 가능한 빨리 벽지를 조심스럽게 떼어내어 재조정하시는 것이...","벽지를 잘못 붙인 경우에는 벽지를 조심스럽게 떼어내어 재조정하는 것뿐만 아니라, 벽...","벽지를 재조정할 때는 벽지를 완전히 제거하고, 벽면을 청소하여 먼지와 기름기를 제거...",벽지를 재조정할 때에는 먼저 부착된 벽지를 조심스럽게 제거하는 것이 중요합니다. 벽...,"벽지를 재조정하는 과정에서는 벽면의 표면을 깔끔하게 정리하고, 필요한 경우 보정 작..."
3,가구수정,벽지 붙이기 전에 뭐 준비해야 돼?,벽지 붙이기 전에 무엇을 준비해야 하나요?,"벽지를 붙이기 전에는 벽의 상태를 점검하고, 필요한 경우 벽을 매끄럽게 하고, 청...","벽지를 부착하기 전에는 벽의 표면을 철저히 점검하여 부식, 균열, 또는 기타 결함...",벽지를 부착하기 전에는 벽의 표면을 철저히 검토하여 과거의 부착물이나 다른 불순물이...,벽지를 부착하기 전에는 벽의 표면을 부드럽게 갈거나 수평을 맞추어야 합니다. 벽면의...,벽지를 부착하기 전에는 벽의 표면을 깨끗하게 닦아야 합니다. 벽면에 먼지나 기름기가...
4,가구수정,벽지 패턴이 제대로 안 맞을 때는 어떡해야 해?,벽지 패턴이 정확하게 맞지 않을 때는 어떻게 해야 하나요?,"벽지 패턴이 맞지 않는 경우, 패턴이 시작되는 지점을 정확하게 측정하고, 패턴을 ...",벽지 패턴이 맞지 않을 때는 패턴의 일치 여부를 확인하고 추가적인 조정이 필요한 부...,"벽지 패턴이 일치하지 않는 경우, 패턴을 조정하는 데에는 주의가 필요합니다. 먼저 ...","벽지 패턴이 맞지 않는 경우, 정확한 패턴 일치를 위해 조정이 필요합니다. 패턴이 ...",벽지 패턴이 맞지 않을 때에는 조정 작업이 필요합니다. 패턴이 시작되는 위치를 정확...
5,가구수정,벽지 붙일 때 가장 중요한 건 뭐라고 생각해?,벽지를 붙일 때 가장 중요한 점은 무엇인가요?,"벽지를 붙일 때는 정확한 위치 선정, 패턴의 일치, 공기방울 없이 매끄럽게 붙이는...","벽지를 부착할 때는 패턴의 일치뿐만 아니라, 공기방울이 없이 매끄럽게 붙이는 것도 ...",벽지를 부착할 때는 패턴의 일치와 더불어 공기방울이 생기지 않도록 주의해야 합니다....,벽지를 부착할 때는 정확한 위치를 선정하여 패턴을 일치시키는 것이 중요합니다. 또한...,"벽지를 부착할 때는 정확한 위치를 선택하고, 패턴을 일치시키는 것뿐만 아니라 공기방..."
6,가구수정,벽지가 물결치거나 주름이 생겼을 때 어떻게 하지?,벽지가 물결치거나 주름이 질 때는 어떻게 해야 하나요?,"벽지가 물결치거나 주름이 질 경우, 롤러나 스퀴지를 사용하여 부드럽게 문질러 주름...","벽지가 물결치거나 주름이 생길 경우, 롤러나 스퀴지를 사용하여 부드럽게 문질러야 합...","벽지가 물결치거나 주름이 생길 경우, 부드럽게 문질러 주름과 공기방울을 제거해야 합...","벽지가 물결치거나 주름이 생기면, 롤러나 스퀴지를 사용하여 부드럽게 문질러야 합니다...","벽지가 물결치거나 주름이 생길 경우, 롤러나 스퀴지를 사용하여 부드럽게 문질러야 합..."
7,가구수정,벽지 붙일 때 기포 어떻게 막지?,벽지를 붙일 때 기포가 생기는 것을 어떻게 방지하나요?,"벽지를 붙일 때 기포가 생기지 않도록, 벽지를 붙이면서 중앙부터 바깥쪽으로 공기를...",벽지를 붙일 때 기포가 생기지 않도록 중앙부터 바깥쪽으로 공기를 밀어내듯이 부드럽게...,벽지를 붙일 때 기포가 생기지 않도록 중앙부터 바깥쪽으로 공기를 밀어내듯이 부드럽게...,벽지를 붙일 때 기포가 생기지 않도록 중앙부터 바깥쪽으로 공기를 밀어내듯이 부드럽게...,벽지를 붙일 때 기포가 생기지 않도록 중앙부터 바깥쪽으로 공기를 밀어내듯이 부드럽게...
8,가구수정,벽지 붙이는데 필요한 도구는 뭐 있어?,벽지를 붙이는데 필요한 도구는 무엇이 있나요?,"벽지를 붙이는데는 벽지용 붓, 롤러, 커터, 스퀴지, 수평계, 줄자 등이 필요합니다.","벽지를 붙일 때는 벽지용 붓은 크고 작은 면적에 따라 적합한 크기를 선택하고, 롤러...","벽지를 붙일 때 벽지용 붓은 모서리와 선명한 부분에 사용되며, 롤러는 큰 면적에 벽...","벽지를 붙일 때 벽지용 붓은 세부 부분에 사용되며, 롤러는 큰 면적에 벽지를 부드럽...","벽지를 붙일 때 벽지용 붓은 세부적인 부분에 사용되며, 롤러는 큰 면적에 벽지를 균..."
9,가구수정,벽지 붙일 때 얼마나 여유분 둬야 돼?,벽지를 붙일 때 얼마나 많은 여유분을 두어야 하나요?,벽지를 붙일 때는 상단과 하단에 최소 5cm 이상의 여유분을 두어 잘림과 오차를 ...,벽지를 붙일 때는 상단과 하단에 최소 5cm 이상의 여유분을 두어 잘림과 오차를 방...,벽지를 붙일 때 상단과 하단에 최소 5cm 이상의 여유분을 두어 잘림과 오차를 방지...,벽지를 붙일 때는 벽의 상단과 하단에 최소 5cm 이상의 여유분을 남겨야 합니다. ...,벽지를 붙일 때는 상단과 하단에 최소 5cm 이상의 여유분을 두어야 합니다. 이렇게...


In [ ]:
# Create empty lists to store your data
questions = []
answers = []

# Iterate through rows of the DataFrame
for index, row in df_df.iterrows():
    # Iterate through '질문' columns
    for q_col in ['질문1', '질문2']:
        # Iterate through '답변' columns
        for a_col in ['답변1', '답변2', '답변3', '답변4', '답변5']:
            # Combine question and answer with </s> token
            input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            # Tokenize the combined text
            input_ids = tokenizer.encode(input_text, return_tensors='pt')

            # Append id, category, question, and answer to the lists
            questions.append(row[q_col])
            answers.append(row[a_col])

# Create a new DataFrame with the combined data
new_df_df = pd.DataFrame({'Question': questions, 'Answer': answers})

In [ ]:
new_df_df.head()

,Question,Answer
0,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,"벽지를 정확한 위치에 붙이기 위해선 먼저 벽의 수평과 수직을 확인하고, 시작점을 ..."
1,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,벽지를 붙일 때는 벽지의 패턴이나 디자인이 맞도록 조심스럽게 배치하는 것이 중요합니...
2,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,벽지를 부착할 때는 시작점을 정한 후에 벽의 수평과 수직을 확인하는 것이 중요합니다...
3,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,벽지를 부착할 때는 먼저 벽의 수평과 수직을 정확히 확인한 후에 시작점을 결정해야 ...
4,벽지를 붙일 때 정확한 위치는 어떻게 찾지?,벽지를 붙이기 전에는 레이저 수평계나 수평 앱을 사용하여 벽의 수평과 수직을 정밀하...


In [ ]:
len(new_df_df)

990

In [ ]:
df = pd.concat([new_df, new_df_df], ignore_index=True)

In [ ]:
len(df)

7430

In [ ]:
df.to_csv('/content/drive/MyDrive/DBDBDeep/last_df.csv', index=False)
df.tail(30)

,Question,Answer
7400,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까?,"벽지면과 벽지면 사이가 벌어진 경우, 건축 전문가나 구조 공학자의 도움이 필요할 수..."
7401,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까?,벽지면과 벽지면 사이가 벌어졌을 때는 먼저 집의 구조에 대한 이해가 필요합니다. 구...
7402,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까?,벽지면과 벽지면 사이가 벌어졌는데 벽체의 수축이나 건축 재료의 변형으로 인한 것으로...
7403,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까?,"벽지면과 벽지면 사이가 벌어졌을 때, 주변 환경에 따라 습기, 온도, 땅의 침하 등..."
7404,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까?,"벽지면과 벽지면 사이가 벌어졌을 때, 건축 디자인의 결함 또는 잘못된 시공이 원인일..."
7405,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까요?,"벽지면과 벽지면 사이가 벌어진 경우, 건축 전문가나 구조 공학자의 도움이 필요할 수..."
7406,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까요?,벽지면과 벽지면 사이가 벌어졌을 때는 먼저 집의 구조에 대한 이해가 필요합니다. 구...
7407,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까요?,벽지면과 벽지면 사이가 벌어졌는데 벽체의 수축이나 건축 재료의 변형으로 인한 것으로...
7408,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까요?,"벽지면과 벽지면 사이가 벌어졌을 때, 주변 환경에 따라 습기, 온도, 땅의 침하 등..."
7409,벽지면과 벽지면 사이가 벌어졌을 때는 어떤 전문가의 도움이 필요할까요?,"벽지면과 벽지면 사이가 벌어졌을 때, 건축 디자인의 결함 또는 잘못된 시공이 원인일..."
